# 계류 법안 분석 

작성자: 박하람

In [1]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [3]:
bill_df = pd.read_csv('bill_20th_data_final.csv', encoding='utf-8', parse_dates=['제안일자','의결일자'])
bill_df = bill_df[['의안번호','의안명','제안자구분','제안일자','의결일자','의결결과','제안회기','제안이유','소관위원회','법률반영여부']]

In [4]:
bill_df.shape

(23684, 10)

In [5]:
bill_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23684 entries, 0 to 23683
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   의안번호    23684 non-null  int64         
 1   의안명     23684 non-null  object        
 2   제안자구분   23684 non-null  object        
 3   제안일자    23684 non-null  datetime64[ns]
 4   의결일자    23684 non-null  datetime64[ns]
 5   의결결과    23684 non-null  object        
 6   제안회기    23684 non-null  object        
 7   제안이유    23016 non-null  object        
 8   소관위원회   23454 non-null  object        
 9   법률반영여부  23684 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 1.8+ MB


In [6]:
# NaN 채우기 
bill_df = bill_df.fillna('')
bill_df.isnull().sum()

의안번호      0
의안명       0
제안자구분     0
제안일자      0
의결일자      0
의결결과      0
제안회기      0
제안이유      0
소관위원회     0
법률반영여부    0
dtype: int64

## EDA 

### 법률: 반영/미반영 기준으로 소요시간 분석

- 법률반영: 원안가결, 원안수정, 대안반영, 수정안반영
- 법률미반영: 부결, 폐기, 철회, 반려, 기타 

In [7]:
bill_df['의결결과'].unique()

array(['임기만료폐기', '원안가결', '가결', '수정가결', '대안반영폐기', '철회', '폐기', '부결',
       '수정안반영폐기', '심사대상제외'], dtype=object)

- accept_list = ['대안반영폐기', '원안가결', '수정가결', '수정안반영폐기', '가결']
- reject_list = ['임기만료폐기', '부결', '철회', '반려', '심사대상제외', '폐기']

In [8]:
bill_tmp = bill_df['의결결과'].value_counts()
bill_tmp

임기만료폐기     14847
대안반영폐기      5135
원안가결        2388
수정가결         898
철회           225
폐기           144
수정안반영폐기       41
부결             3
심사대상제외         2
가결             1
Name: 의결결과, dtype: int64

In [9]:
bill_df['법률반영여부'].value_counts()

미반영    15221
반영      8463
Name: 법률반영여부, dtype: int64

In [10]:
bill_df['의결처리기간'] = (bill_df['의결일자'] - bill_df['제안일자']).dt.days
bill_df.head(3)

,의안번호,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,법률반영여부,의결처리기간
0,2024996,집합건물의 소유 및 관리에 관한 법률 일부개정법률안(김병관의원 등 13인),의원,2020-05-22,2020-05-29,임기만료폐기,제20대 (2016~2020) 제378회,아파트 등 공동주택은 「공동주택관리법」에 의해 체계적으로 관리되고 있는 반면 ...,법제사법위원회,미반영,7
1,2024995,지방세법 일부개정법률안(김병관의원 등 10인),의원,2020-05-22,2020-05-29,임기만료폐기,제20대 (2016~2020) 제378회,현행법상 아파트 등 주택에 대한 재산세 과세표준은 국토교통부가 해마다 「부동산 가격...,행정안전위원회,미반영,7
2,2024994,법률용어 정비를 위한 정보위원회 소관 2개 법률 일부개정을 위한 법률안,위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,,정보위원회,반영,0


In [11]:
bill_df['의결처리기간'].groupby(bill_df['법률반영여부']).mean()

법률반영여부
미반영    760.262072
반영     245.408720
Name: 의결처리기간, dtype: float64

In [12]:
pend_df = pd.pivot_table(bill_df, 
                        index=['법률반영여부', '의결결과'],
                        values=['의결처리기간']).fillna(0)
pend_df

의결처리기간
법률반영여부 의결결과               
미반영    부결       150.666667
       심사대상제외   829.000000
       임기만료폐기   774.433960
       철회        91.528889
       폐기       355.722222
반영     가결         0.000000
       대안반영폐기   325.360273
       수정가결     287.788419
       수정안반영폐기  207.975610
       원안가결      58.294807

In [13]:
# 의결처리기간 - 평균 
pd.merge(pend_df, bill_tmp, left_on='의결결과', right_index=True)

의결처리기간   의결결과
법률반영여부 의결결과                      
미반영    부결       150.666667      3
       심사대상제외   829.000000      2
       임기만료폐기   774.433960  14847
       철회        91.528889    225
       폐기       355.722222    144
반영     가결         0.000000      1
       대안반영폐기   325.360273   5135
       수정가결     287.788419    898
       수정안반영폐기  207.975610     41
       원안가결      58.294807   2388

In [14]:
print('미반영 평균 소요시간: ', pend_df.loc['미반영','의결처리기간'].mean())
print('반영 평균 소요시간: ', pend_df.loc['반영','의결처리기간'].mean())

미반영 평균 소요시간:  440.2703475000561
반영 평균 소요시간:  175.88382169465518


## 소관위원회별 토픽 모델링 



In [27]:
comm_df = bill_df.copy()

국회법 제4장 회의 제2절 발의, 동의, 철회와 번안 

> 제33조 3번 의안이 발의 또는 제출되었을 때에는 의장은 이것을 국회에 보고한 후 적당한 위원회에 부탁하고 그 심사가 끝난 뒤에 본회의에 부의한다. **단, 법률안 이외의 의안은 국회의 결의에 의하여 위원회의 심사를 생략할 수 있다.**

In [57]:
# 소관위원회가 없는 경우 임명안이거나 대다수 의원들이 발의한 법안임. 
# 따로 분류를 해줄 필요가 있다고 생각했음.
comm_df.loc[comm_df['소관위원회'] == '','소관위원회'] = '기타'
comm_df[comm_df['소관위원회'] == '기타'].head(3)

,의안번호,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,법률반영여부,의결처리기간,위원회 분류
60,2024936,국가인권위원회 위원(석원정) 선출안,의장,2020-05-20,2020-05-20,가결,제20대 (2016~2020) 제378회,,기타,반영,0,상임위원회
71,2024928,2019년도 국정감사 결과보고서 채택의 건,위원장,2020-05-06,2020-05-20,원안가결,제20대 (2016~2020) 제377회,,기타,반영,14,상임위원회
74,2024922,공휴일 본회의 개의에 관한 건,의장,2020-04-29,2020-04-29,원안가결,제20대 (2016~2020) 제377회,,기타,반영,0,상임위원회


In [58]:
comm_df['소관위원회'].value_counts()

행정안전위원회                          2751
보건복지위원회                          2528
국토교통위원회                          2203
기획재정위원회                          2047
환경노동위원회                          2018
법제사법위원회                          1899
농림축산식품해양수산위원회                    1668
정무위원회                            1612
산업통상자원중소벤처기업위원회                  1264
과학기술정보방송통신위원회                     990
교육위원회                             923
문화체육관광위원회                         792
국회운영위원회                           613
국방위원회                             547
여성가족위원회                           403
외교통일위원회                           376
교육문화체육관광위원회                       247
기타                                230
정치개혁 특별위원회                        145
안전행정위원회                           144
산업통상자원위원회                          99
헌법개정 및 정치개혁 특별위원회                  51
윤리특별위원회                            43
정보위원회                              35
사법개혁 특별위원회                         30
미래창조과학방송통신위원회                      12
예산결산특별위원회   

In [59]:
comm = comm_df['소관위원회'].unique()
special_comm = [com for com in comm if com.endswith('특별위원회') == True]
basic_comm = [com for com in comm if com.endswith('특별위원회') == False]

In [60]:
print('상설 특별위원회 리스트:', '\n', special_comm, '\n')
print('상임위원회 리스트:', '\n', basic_comm)

상설 특별위원회 리스트: 
 ['국회 코로나19 대책 특별위원회', '헌법개정 및 정치개혁 특별위원회', '중앙선거관리위원회 위원 선출에 관한 인사청문특별위원회', '예산결산특별위원회', '정치개혁 특별위원회', '사법개혁 특별위원회', '윤리특별위원회', '헌법재판소 재판관 선출에 관한 인사청문특별위원회', '미세먼지 대책 특별위원회', '평창동계올림픽 및 국제경기대회지원 특별위원회', '저출산·고령화대책 특별위원회', '국회상임위원회 위원정수에 관한 규칙 개정 특별위원회'] 

상임위원회 리스트: 
 ['법제사법위원회', '행정안전위원회', '정보위원회', '여성가족위원회', '문화체육관광위원회', '국회운영위원회', '과학기술정보방송통신위원회', '정무위원회', '기획재정위원회', '국토교통위원회', '환경노동위원회', '기타', '국방위원회', '외교통일위원회', '농림축산식품해양수산위원회', '산업통상자원중소벤처기업위원회', '교육위원회', '보건복지위원회', '교육문화체육관광위원회', '안전행정위원회', '산업통상자원위원회', '미래창조과학방송통신위원회']


In [33]:
comm_df.loc[comm_df['소관위원회'].isin(basic_comm), '위원회 분류'] = '상임위원회'
comm_df.loc[comm_df['소관위원회'].isin(special_comm), '위원회 분류'] = '특별위원회'

### 상임위원회 분석

In [76]:
comm_df1 = comm_df[comm_df['위원회 분류'] == '상임위원회']
comm_df1.head(3)

,의안번호,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,법률반영여부,의결처리기간,위원회 분류
0,2024996,집합건물의 소유 및 관리에 관한 법률 일부개정법률안(김병관의원 등 13인),의원,2020-05-22,2020-05-29,임기만료폐기,제20대 (2016~2020) 제378회,아파트 등 공동주택은 「공동주택관리법」에 의해 체계적으로 관리되고 있는 반면 ...,법제사법위원회,미반영,7,상임위원회
1,2024995,지방세법 일부개정법률안(김병관의원 등 10인),의원,2020-05-22,2020-05-29,임기만료폐기,제20대 (2016~2020) 제378회,현행법상 아파트 등 주택에 대한 재산세 과세표준은 국토교통부가 해마다 「부동산 가격...,행정안전위원회,미반영,7,상임위원회
2,2024994,법률용어 정비를 위한 정보위원회 소관 2개 법률 일부개정을 위한 법률안,위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,,정보위원회,반영,0,상임위원회


In [73]:
comm_df1['소관위원회'].value_counts()

행정안전위원회            2751
보건복지위원회            2528
국토교통위원회            2203
기획재정위원회            2047
환경노동위원회            2018
법제사법위원회            1899
농림축산식품해양수산위원회      1668
정무위원회              1612
산업통상자원중소벤처기업위원회    1264
과학기술정보방송통신위원회       990
교육위원회               923
문화체육관광위원회           792
국회운영위원회             613
국방위원회               547
여성가족위원회             403
외교통일위원회             376
교육문화체육관광위원회         247
기타                  230
안전행정위원회             144
산업통상자원위원회            99
정보위원회                35
미래창조과학방송통신위원회        12
Name: 소관위원회, dtype: int64

In [82]:
def split_name(x): 
    name = x.split('(')[0]
    return name

In [83]:
comm_df1['의안명 전처리'] = comm_df1['의안명'].apply(lambda x: split_name(x))

<ipython-input-83-2331eb945b2b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comm_df1['의안명 전처리'] = comm_df1['의안명'].apply(lambda x: split_name(x))


In [109]:
comm_df1['의안명+제안이유'] = (comm_df1['의안명 전처리'] + ' ' + comm_df1['제안이유']).str.strip()

<ipython-input-109-b7cc7a351d1f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comm_df1['의안명+제안이유'] = (comm_df1['의안명 전처리'] + ' ' + comm_df1['제안이유']).str.strip()


In [159]:
comm_anal1 = comm_df1[['소관위원회','의결결과','법률반영여부','의결처리기간','의안명+제안이유']]

for index, com in enumerate(basic_comm): 
    print(index, com)

0 법제사법위원회
1 행정안전위원회
2 정보위원회
3 여성가족위원회
4 문화체육관광위원회
5 국회운영위원회
6 과학기술정보방송통신위원회
7 정무위원회
8 기획재정위원회
9 국토교통위원회
10 환경노동위원회
11 기타
12 국방위원회
13 외교통일위원회
14 농림축산식품해양수산위원회
15 산업통상자원중소벤처기업위원회
16 교육위원회
17 보건복지위원회
18 교육문화체육관광위원회
19 안전행정위원회
20 산업통상자원위원회
21 미래창조과학방송통신위원회


#### [0] 법제사법위원회

In [133]:
com1 = comm_anal1[comm_anal1['소관위원회'] == basic_comm[0]].reset_index()

In [150]:
def preprocessing(text):
    # 특수문자 제거
    text = re.sub('[?.,;:|\)(*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

com1['의안명+제안이유'] = com1['의안명+제안이유'].apply(lambda x: preprocessing(x))